In [ ]:
while True:
    import os
    import subprocess

    from rp import *

    sys.path += rp.get_absolute_paths(
        [
            # "~/CleanCode/Management",
            "~/CleanCode/Github/DiffusionAsShader",
            # "~/CleanCode/Datasets/Vids/Raw_Feb28",
            # "~/CleanCode/Github/CogvideX-Interpolation-Mar23:MotionPrompting",
            # "~/CleanCode/Github/CogvideX-Interpolation-Feb13:Inpainting",
        ]
    )

    from source.datasets.youtube.youtube_dataset import ProcessedYoutubeDataset

    sleep(random_float())
    device = rp.select_torch_device(reserve=True, prefer_used=True)
    set_cuda_visible_devices(device)

    dataset = ProcessedYoutubeDataset()
    sample = dataset[123]
    sample = random_element(dataset)

    prompt = sample.prompt
    input_video_path = sample.video_path
    tracking_video_path = sample.video_dasTrackvid_path

    def resize_video_file(video_path):
        """Takes an input MP4, and returns a temp 480x720 MP4 with 49 frames"""
        return rp.save_video_mp4(
            rp.resize_list(
                rp.resize_images(rp.load_video(video_path), size=(480, 720)),
                49,
            ),
            rp.temporary_file_path("mp4"),
            video_bitrate="max",
        )

    temp_input_video_path = resize_video_file(input_video_path)
    temp_tracking_video_path = resize_video_file(input_video_path)

    def run_diffusion_as_shader_subprocess(
        prompt="A ski lift is going up a snowy mountain in the alps with a few houses on it.",
        input_video_path="/home/jupyter/CleanCode/Datasets/Vids/Raw_Feb28/Processed_April7/--_vV8VBXpc_13387073_34287675/video.mp4",
        tracking_video_path="/home/jupyter/CleanCode/Datasets/Vids/Raw_Feb28/Processed_April7/--_vV8VBXpc_13387073_34287675/video.mp4__DiffusionAsShaderCondition/tracking_video.mp4",
        num_inference_steps=25,
    ):
        output_name = f"dasOut_steps={num_inference_steps}"
        output_path = get_unique_copy_path(
            with_file_name(input_video_path, output_name)
        )

        if path_exists(output_path):
            return output_path

        _, height, width, _ = rp.get_video_file_shape(tracking_video_path)
        input_image = next(load_video_stream(input_video_path))
        input_image = rp.cv_resize_image(input_image, (height, width))
        input_image = save_image(input_image, temporary_file_path())

        work_dir = "/home/jupyter/CleanCode/Github/DiffusionAsShader"
        model_path = "./diffusion_shader_model"

        subprocess.run(
            [
                # fmt: off
                sys.executable,
                'testing/inference.py',
                '--prompt', prompt,
                '--model_path', model_path,
                '--output_path', output_path,
                '--tracking_path', tracking_video_path,
                '--image_or_video', input_image,
                '--num_inference_steps', str(num_inference_steps),
                '--generate_type', 'i2v',
                # fmt: on
            ],
            shell=False,
            cwd=work_dir,
        )

        return output_path

    output_video_path = run_diffusion_as_shader_subprocess(
        prompt,
        temp_input_video_path,
        temp_tracking_video_path,
    )

    preview_video = rp.labeled_images(
        rp.horizontally_concatenated_videos(
            rp.labeled_videos(
                rp.load_videos(
                    [
                        temp_input_video_path,
                        output_video_path,
                        tracking_video_path,
                    ]
                ),
                [
                    "Input Video",
                    "Output Video",
                    "Tracks Video",
                ],
                text_color = 'white blue',
                font="R:Futura",
                size=30,
            )
        ),
        "Diffusion As Shader (Vanilla)",
        font="R:Futura",
        size=40,
    )
    rp.display_video(preview_video)
    save_video_mp4(preview_video, rp.get_unique_copy_path("preview_video.mp4"))

    rp.delete_files(
        temp_input_video_path,
        temp_tracking_video_path,
        strict=False,
    )

    # sample.release()
